In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [2]:
processed_df = pd.read_pickle('../data/preprocessed_df.pkl')

In [3]:
processed_df.head()

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text
0,1.502177e+09,840.0,TheRedPill,0.0,0.65,https://www.reddit.com/r/TheRedPill/comments/6...,False,Google Fires Ph.D Biologist/Engineer For Claim...,"[google, fire, ph.d, biologist/engineer, claim...",google fire ph.d biologist/engineer claiming a...
1,1.480424e+09,759.0,TheRedPill,0.0,0.82,https://www.reddit.com/r/TheRedPill/comments/5...,False,HOW TO GET LAID LIKE A WARLORD: 37 Rules of Ap...,"[laid, warlord, 37, rule, approaching, model-t...",laid warlord 37 rule approaching model-tier gi...
2,1.502101e+09,179.0,TheRedPill,0.0,0.86,https://www.reddit.com/r/TheRedPill/comments/6...,False,I have been practicing Stoicism for 3 years no...,"[practicing, stoicism, 3, year, quality, life,...",practicing stoicism 3 year quality life increa...
3,1.433962e+09,954.0,TheRedPill,0.0,0.87,https://www.reddit.com/r/TheRedPill/comments/3...,False,"[META] Reddit rolls out first ban wave of ""Har...","[meta, reddit, roll, first, ban, wave, harassi...",meta reddit roll first ban wave harassing subr...
4,1.500270e+09,336.0,TheRedPill,0.0,0.90,https://www.reddit.com/r/TheRedPill/comments/6...,False,"Man gets his ex-GF to pay child support, and R...","[man, get, ex-gf, pay, child, support, reddit,...",man get ex-gf pay child support reddit freak o...


In [4]:
tfidf_vectorizer = TfidfVectorizer(max_df = .95)

text_transformed = tfidf_vectorizer.fit_transform(processed_df['finalized_text'])

In [5]:
text_transformed

<11009x27094 sparse matrix of type '<class 'numpy.float64'>'
	with 182287 stored elements in Compressed Sparse Row format>

In [6]:
k_clusterer = KMeans(n_clusters = 10, random_state = 305)

k_clusterer.fit(text_transformed)

KMeans(n_clusters=10, random_state=305)

In [7]:
processed_df['k_means_label'] = k_clusterer.labels_

In [8]:
processed_df['k_means_label'].value_counts()

4    8320
0     947
2     469
3     299
9     294
7     214
5     210
8     142
6      72
1      42
Name: k_means_label, dtype: int64

In [9]:
processed_df

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text,k_means_label
0,1.502177e+09,840.0,TheRedPill,0.0,0.65,https://www.reddit.com/r/TheRedPill/comments/6...,False,Google Fires Ph.D Biologist/Engineer For Claim...,"[google, fire, ph.d, biologist/engineer, claim...",google fire ph.d biologist/engineer claiming a...,2
1,1.480424e+09,759.0,TheRedPill,0.0,0.82,https://www.reddit.com/r/TheRedPill/comments/5...,False,HOW TO GET LAID LIKE A WARLORD: 37 Rules of Ap...,"[laid, warlord, 37, rule, approaching, model-t...",laid warlord 37 rule approaching model-tier gi...,2
2,1.502101e+09,179.0,TheRedPill,0.0,0.86,https://www.reddit.com/r/TheRedPill/comments/6...,False,I have been practicing Stoicism for 3 years no...,"[practicing, stoicism, 3, year, quality, life,...",practicing stoicism 3 year quality life increa...,0
3,1.433962e+09,954.0,TheRedPill,0.0,0.87,https://www.reddit.com/r/TheRedPill/comments/3...,False,"[META] Reddit rolls out first ban wave of ""Har...","[meta, reddit, roll, first, ban, wave, harassi...",meta reddit roll first ban wave harassing subr...,5
4,1.500270e+09,336.0,TheRedPill,0.0,0.90,https://www.reddit.com/r/TheRedPill/comments/6...,False,"Man gets his ex-GF to pay child support, and R...","[man, get, ex-gf, pay, child, support, reddit,...",man get ex-gf pay child support reddit freak o...,2
...,...,...,...,...,...,...,...,...,...,...,...
11004,1.633457e+09,900.0,Anarcho_Capitalism,186148.0,0.88,https://v.redd.it/0ptsjdwh9or71,False,"The is Los Angeles, California","[los, angeles, california]",los angeles california,4
11005,1.632157e+09,2229.0,Anarcho_Capitalism,186148.0,0.83,https://i.redd.it/xmgcuoawtoo71.png,True,Personal freedoms,"[personal, freedom]",personal freedom,4
11006,1.591131e+09,141.0,Anarcho_Capitalism,186148.0,0.93,https://i.imgur.com/ticpuuy.jpg,True,Progressive Liberal,"[progressive, liberal]",progressive liberal,4
11007,1.638208e+09,506.0,Anarcho_Capitalism,186148.0,0.89,https://i.redd.it/dcyjbpirmk281.jpg,True,Welp…,[welp…],welp…,4


In [17]:
agg_clusterer = AgglomerativeClustering(n_clusters = 100, compute_full_tree = True, compute_distances = True)
agg_clusterer.fit(text_transformed.toarray())

AgglomerativeClustering(compute_distances=True, compute_full_tree=True)

In [18]:
processed_df['agg_labels'] = agg_clusterer.labels_

In [19]:
processed_df['agg_labels'].value_counts()

0    10982
1       27
Name: agg_labels, dtype: int64

In [16]:
processed_df.loc[processed_df['agg_labels'] == 9]

,created_utc,num_comments,subreddit,subreddit_subscribers,upvote_ratio,url,image?,total_text,tokenned_stopped_text,finalized_text,k_means_label,agg_labels
1625,1.623692e+09,3992.0,mildlyinfuriating,4234469.0,0.84,https://i.redd.it/qixmzslho9571.png,True,What the actual fuck?,[fuck],fuck,4,9
6150,1.652875e+09,390.0,MarchAgainstNazis,114804.0,0.93,https://i.redd.it/ybg8aqiu58091.png,True,Fuck them and fuck them hard,"[fuck, fuck, hard]",fuck fuck hard,4,9
6933,1.498603e+09,928.0,EnoughTrumpSpam,99619.0,0.75,https://i.redd.it/tt140nenl96z.png,True,Fuck This Guy,"[fuck, guy]",fuck guy,0,9
7005,1.592580e+09,1734.0,ABoringDystopia,716734.0,0.91,https://i.redd.it/1pn97476vv551.jpg,True,fuck me,[fuck],fuck,4,9
7018,1.616606e+09,997.0,ABoringDystopia,716734.0,0.88,https://i.redd.it/y7czf71pc0p61.png,True,Fuck her.,[fuck],fuck,4,9
7310,1.605712e+09,916.0,TheRightCantMeme,401655.0,0.87,https://i.redd.it/6yu829ocj0061.jpg,True,What the fuck,[fuck],fuck,4,9
7670,1.595416e+09,27.0,TheBluePill,96486.0,0.97,https://i.redd.it/6gndgfky4ec51.jpg,True,Oh fuck off,[fuck],fuck,4,9
7734,1.511696e+09,705.0,justneckbeardthings,884942.0,0.95,https://i.redd.it/iq8m1d264b001.png,True,Fuck you then,[fuck],fuck,4,9
8399,1.585628e+09,706.0,JoeRogan,769785.0,0.90,https://i.redd.it/si0n2klqwvp41.jpg,True,Fuck off,[fuck],fuck,4,9
10668,1.638140e+09,16.0,libsofreddit,18024.0,0.98,https://i.redd.it/k3x0pw0s0f281.jpg,True,Fuck both of them,[fuck],fuck,4,9
